# Statistics and Data Manipulation
([Back to Overview](../index.html#/0/4))

In [1]:
import Base: filter, map
filter(f::Function)::Function = x -> filter(f, x)
map(f::Function)::Function    = x -> map(f, x)

map (generic function with 61 methods)

In [2]:
x = collect(range(0, 10))

11-element Vector{Int64}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [3]:
x |> map(x->x+1)

11-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [4]:
x |> map(x->x+1) |> filter(x->x%2==0)

5-element Vector{Int64}:
  2
  4
  6
  8
 10